In [1]:
!pip install fastembed llama-index llama-index-embeddings-huggingface llama-index-postprocessor-flag-embedding-reranker llama-index-vector-stores-qdrant matplotlib numpy openai pandas python-dotenv sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.1/282.1 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 114.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
os.chdir("/content/drive/MyDrive/ALQAC")

In [2]:
from llama_index.core.schema import (
    TextNode,
    NodeRelationship,
    RelatedNodeInfo,
    ObjectType,
)
from typing import List, Dict
import uuid
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from law_search.vector_db import QdrantCollection
from llama_index.core import Settings

# Hardcoded configurations
MODEL_NAME = "anhtld/VN-Law-Embedding"
# MODEL_CACHE_DIR = "./models"
COLLECTION_NAME = "law_sections"


def process_json_content(json_content: List[Dict], file_name: str) -> List[TextNode]:
    """
    Process JSON content and create TextNodes with relationships.

    Args:
        json_content: List containing the document json
        file_name: Name of the source file

    Returns:
        List of TextNodes with established relationships
    """

    nodes = []
    for content in json_content:
        section_id = list(content.keys())[0]
        section_data = list(content.values())[0]
        # Create text node
        node = TextNode(
            text=section_data,
            id_=str(uuid.uuid4()),
            metadata={
                "doc_id": file_name,
                "section_id": section_id,
                "title": section_data.split("\n\n")[0],
            },
        )
        nodes.append(node)

    for i, node in enumerate(nodes):
        if i > 0:
            node.relationships[NodeRelationship.PREVIOUS] = RelatedNodeInfo(
                node_id=nodes[i - 1].node_id,
                node_type=ObjectType.TEXT,
                hash=nodes[i - 1].hash,
            )
        if i < len(nodes) - 1:
            node.relationships[NodeRelationship.NEXT] = RelatedNodeInfo(
                node_id=nodes[i + 1].node_id,
                node_type=ObjectType.TEXT,
                hash=nodes[i + 1].hash,
            )
    return nodes


def setup_embedding_model() -> None:
    """Initialize and setup the embedding model."""

    embed_model = HuggingFaceEmbedding(
        model_name=MODEL_NAME,
        trust_remote_code=True,
        # cache_folder=MODEL_CACHE_DIR,
    )
    Settings.embed_model = embed_model
    Settings.chunk_size = 512
    Settings.db = QdrantCollection(collection_name="law_sections")

In [3]:
# Setup embedidng
setup_embedding_model()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Initializing Qdrant with data directory: /content/drive/MyDrive/ALQAC/qdrant
Vector store initialized successfully


Only run once

In [4]:
from pathlib import Path
import json
from law_search import QdrantCollection

nodes = []
# Indexing to qdrant local db

output_path = Path("./output")
for filename in output_path.glob("*"):
    print(filename.stem)
    with open(filename, "r") as file:
        json_content = json.load(file)
    nodes.extend(process_json_content(json_content, filename.stem))

Settings.db.insert_nodes(nodes)

Luật Trồng trọt
Luật Chăn nuôi
Luật Điện ảnh
Luật An ninh mạng
Luật Phòng, chống ma túy
Luật Bảo vệ môi trường
Luật Giáo dục
Bộ luật dân sự
Hiến pháp
Luật Đường bộ
Luật Tài nguyên nước
Luật Tiếp cận thông tin
Luật Cư trú
Luật Tố tụng hành chính
Luật Trọng tài thương mại
Luật Giao dịch điện tử
Luật Căn cước
Luật Viên chức
Luật Bảo vệ quyền lợi người tiêu dùng
Luật Hôn nhân và gia đình
Luật Đất đai
Luật Du lịch
Luật Viễn thông
Luật Thanh niên
Luật Giá
Luật Phòng, chống tác hại của rượu, bia
Luật Nhà ở
Luật Khám bệnh, chữa bệnh


/usr/local/lib/python3.11/dist-packages/llama_index/vector_stores/qdrant/base.py:709: UserWarning: Payload indexes have no effect in the local Qdrant. Please use server Qdrant if you need payload indexes.
  self._client.create_payload_index(


# Retriever

In [5]:
TOP_K = 2
SPARSE_TOP_K = 12
retriever_engine = Settings.db._index.as_retriever(
    similarity_top_k=TOP_K,
    sparse_top_k=SPARSE_TOP_K,
    vector_store_query_mode="hybrid",
    node_postprocessor=[],
)


In [6]:
from llama_index.core.schema import QueryBundle


def retrieve(query):
    result_nodes = retriever_engine._retrieve(
        QueryBundle(
            query_str=query,
        )
    )
    result_dict = {"result": []}
    for node in result_nodes:
        if node.score < 0.5:
            if TOP_K > 1:
                continue

        else:
            result_dict["result"].append(
                {
                    "document": node.node.metadata["doc_id"],
                    "id": node.node.metadata["section_id"],
                    "score": node.score,
                    "text": node.node.text,
                }
            )
    return result_dict


query = """
Một người có thể được người có quyền theo quy định của pháp luật yêu cầu Tòa án ra quyết định tuyên bố là đã chết khi người đó biệt tích trong chiến tranh sau 05 năm, kể từ ngày chiến tranh kết thúc mà vẫn không có tin tức xác thực là còn sống, đúng hay sai?
"""
retrieve(query)


{'result': [{'document': 'Bộ luật dân sự',
   'id': '71',
   'score': 1.0,
   'text': 'Tuyên bố chết\n\n1. Người có quyền, lợi ích liên quan có thể yêu cầu Tòa án ra quyết định tuyên bố một người là đã chết trong trường hợp sau đây:\n\na) Sau 03 năm, kể từ ngày quyết định tuyên bố mất tích của Tòa án có hiệu lực pháp luật mà vẫn không có tin tức xác thực là còn sống;\n\nb) Biệt tích trong chiến tranh sau 05 năm, kể từ ngày chiến tranh kết thúc mà vẫn không có tin tức xác thực là còn sống;\n\nc) Bị tai nạn hoặc thảm họa, thiên tai mà sau 02 năm, kể từ ngày tai nạn hoặc thảm hoạ, thiên tai đó chấm dứt vẫn không có tin tức xác thực là còn sống, trừ trường hợp pháp luật có quy định khác;\n\nd) Biệt tích 05 năm liền trở lên và không có tin tức xác thực là còn sống; thời hạn này được tính theo quy định tại khoản 1 Điều 68 của Bộ luật này.\n\n2. Căn cứ vào các trường hợp quy định tại khoản 1 Điều này, Tòa án xác định ngày chết của người bị tuyên bố là đã chết.\n\n3. Quyết định của Tòa 

# Evaluation

In [7]:
# Evaluation
import json

path = "ALQAC_2025_data/alqac25_train.json"

with open(path, "r", encoding="utf-8") as f:
    json_data = json.load(f)

json_data[:2]

[{'question_id': 'train_alqac25_1',
  'question_type': 'Đúng/Sai',
  'text': 'Người nghiện ma túy từ đủ 18 tuổi trở lên bị áp dụng biện pháp xử lý hành chính đưa vào cơ sở cai nghiện bắt buộc theo quy định của Luật Xử lý vi phạm hành chính khi bị phát hiện sử dụng chất ma túy một cách trái phép trong thời gian cai nghiện ma túy tự nguyện, đúng hay sai?',
  'relevant_articles': [{'law_id': 'Luật Phòng, chống ma túy',
    'article_id': '32'}],
  'answer': 'Đúng'},
 {'question_id': 'train_alqac25_2',
  'question_type': 'Đúng/Sai',
  'text': 'Quan hệ hôn nhân và gia đình có yếu tố nước ngoài là quan hệ hôn nhân và gia đình mà ít nhất một bên tham gia là người nước ngoài, người Việt Nam định cư ở nước ngoài, đúng hay sai?',
  'relevant_articles': [{'law_id': 'Luật Hôn nhân và gia đình',
    'article_id': '3'}],
  'answer': 'Đúng'}]

In [8]:
number_of_articles = {}
for item in json_data:
    count = len(item["relevant_articles"])
    if count not in number_of_articles:
        number_of_articles[count] = 1
    else:
        number_of_articles[count] += 1
number_of_articles

{1: 718, 2: 10, 3: 1}

In [9]:
def calculate_precision(retrieved_articles, relevant_articles):
    """
    Calculates precision for a single question.
    A retrieved article is correct if its (law_id, article_id) tuple matches a relevant article.
    """
    retrieved_set = {(item["document"], item["id"]) for item in retrieved_articles}
    relevant_set = {(item["law_id"], item["article_id"]) for item in relevant_articles}

    correctly_retrieved = len(retrieved_set.intersection(relevant_set))
    total_retrieved = len(retrieved_set)

    if total_retrieved == 0:
        return 0.0

    return correctly_retrieved / total_retrieved


def calculate_recall(retrieved_articles, relevant_articles):
    """
    Calculates recall for a single question.
    A retrieved article is correct if its (law_id, article_id) tuple matches a relevant article.
    """
    retrieved_set = {(item["document"], item["id"]) for item in retrieved_articles}
    relevant_set = {(item["law_id"], item["article_id"]) for item in relevant_articles}

    correctly_retrieved = len(retrieved_set.intersection(relevant_set))
    total_relevant = len(relevant_set)

    if total_relevant == 0:
        return 0.0

    return correctly_retrieved / total_relevant


def calculate_f2_score(precision, recall):
    """
    Calculates the F2 score based on the provided formula.
    """
    if (4 * precision + recall) == 0:
        return 0.0

    return (5 * precision * recall) / (4 * precision + recall)


In [10]:
import os
import json

all_precision = []
all_recall = []
all_f2_scores = []
details = []
evaluation_path = "evaluation3"
os.makedirs(evaluation_path, exist_ok=True)

for idx, item in enumerate(json_data):
    query = item["text"]
    relevant_articles = item["relevant_articles"]

    retrieved_results = retrieve(query)
    retrieved_articles = retrieved_results.get("result", [])
    precision = calculate_precision(retrieved_articles, relevant_articles)
    recall = calculate_recall(retrieved_articles, relevant_articles)
    f2 = calculate_f2_score(precision, recall)

    all_precision.append(precision)
    all_recall.append(recall)
    all_f2_scores.append(f2)
    details.append(
        {
            "question_id": item.get("question_id", idx),
            "query": query,
            "precision": precision,
            "recall": recall,
            "f2_score": f2,
            "retrieved_articles": retrieved_articles,
            "relevant_articles": relevant_articles,
        }
    )

if all_f2_scores:
    average_f2 = sum(all_f2_scores) / len(all_f2_scores)
    average_precision = sum(all_precision) / len(all_precision)
    average_recall = sum(all_recall) / len(all_recall)
    print(f"Average Precision: {average_precision:.4f}")
    print(f"Average Recall: {average_recall:.4f}")
    print(f"Average F2-Score: {average_f2:.4f}")
else:
    print("Could not calculate F2-Score, no data processed.")

with open(
    f"{evaluation_path}/detailed_metrics_{TOP_K}_{SPARSE_TOP_K}.json",
    "w",
    encoding="utf-8",
) as f:
    json.dump(
        {
            "average_precision": average_precision if all_f2_scores else None,
            "average_recall": average_recall if all_f2_scores else None,
            "average_f2_score": average_f2 if all_f2_scores else None,
            "details": details,
        },
        f,
        ensure_ascii=False,
        indent=2,
    )


Average Precision: 0.0000
Average Recall: 0.0000
Average F2-Score: 0.0000


In [ ]:
- generate 5 queries -> retriev 5 times -> combine the retrieved_results
- use some examples datasets for style transfer
- apply reranker for post processing